In [1]:
# https://github.com/huggingface/notebooks/blob/master/examples/language_modeling_from_scratch.ipynb
# https://github.com/huggingface/transformers/tree/master/notebooks
# https://huggingface.co/transformers/model_doc/xlnet.html#transformers.XLNetTokenizer 

In [2]:
import os
os.environ['HF_HOME'] = os.path.join(os.getcwd(), 'hf_cache')

from transformers import XLNetConfig, XLNetModel, XLNetTokenizer, XLNetLMHeadModel 
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from tqdm import tqdm

In [3]:
tokenizer = XLNetTokenizer(vocab_file='models/smiles_sp.model',
                           do_lower_case=False,
                           keep_accents=True
                           )

In [4]:
dataset = load_dataset('text', data_files=['data/proc_zinc/all.txt'])

Using custom data configuration default-ce80c9ae12ee94b9
Reusing dataset text (e:\molnlp\mol-prop\hf_cache\datasets\text\default-ce80c9ae12ee94b9\0.0.0\e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 1/1 [00:00<00:00,  4.59it/s]


In [5]:
print(dataset['train'][1]['text'])
print(tokenizer(dataset['train'][1]['text'], padding='max_length', max_length=100))

CC(C)(C)OC(=O)N[C@H]1CONC1=O
{'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 3, 11, 3, 4, 11, 3, 4, 5, 3, 10, 5, 4, 9, 7, 3, 12, 6, 15, 3, 5, 9, 3, 39, 5, 0, 0], 'token_type_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
max_len_path = 'data/proc_zinc/max_len.txt'
if os.path.exists(max_len_path):
    max_len = int(open(max_len_path).read())
else:
    max_len = 0
    with tqdm(total=len(dataset['train'])) as pbar:
        for data in dataset['train']:
            max_len = max(len(data['text']), max_len)
            pbar.update(1)
    with open(max_len_path, 'w') as f:
        f.write(str(max_len))

In [7]:
def tokenize_function(examples):
    out_dict = tokenizer(examples["text"], padding='max_length', max_length=max_len)
    out_dict['labels'] = out_dict['input_ids'].copy()
    return out_dict


tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Loading cached processed dataset at e:\molnlp\mol-prop\hf_cache\datasets\text\default-ce80c9ae12ee94b9\0.0.0\e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5\cache-151da96b9fb5f3c8.arrow


In [8]:
model_config = XLNetConfig(
    vocab_size=tokenizer.vocab_size,
    n_layer=12
)
model = XLNetLMHeadModel(model_config)

In [9]:
max_samples = 200000
batch_size = 10
max_steps = max_samples // batch_size
save_steps = max_steps // 5
training_args = TrainingArguments(
    f"models/xlnet-smiles-bs-{batch_size}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    num_train_epochs=1,
    max_steps = max_steps,
    #fp16=True,
    save_steps=save_steps,
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
)

max_steps is given, it will override any value given in num_train_epochs


In [11]:
trainer.train()

***** Running training *****
  Num examples = 6072715
  Num Epochs = 1
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 20000
  2%|▎         | 500/20000 [04:06<2:27:56,  2.20it/s]

{'loss': 0.0166, 'learning_rate': 1.95e-05, 'epoch': 0.0}


  5%|▌         | 1000/20000 [08:07<2:55:36,  1.80it/s]

{'loss': 0.0005, 'learning_rate': 1.9e-05, 'epoch': 0.0}


  8%|▊         | 1500/20000 [12:35<2:51:46,  1.79it/s]

...

{'loss': 0.0, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.03}


100%|██████████| 20000/20000 [2:47:43<00:00,  2.19it/s]Saving model checkpoint to models/xlnet-smiles\checkpoint-20000
Configuration saved in models/xlnet-smiles\checkpoint-20000\config.json


{'loss': 0.0, 'learning_rate': 0.0, 'epoch': 0.03}


Model weights saved in models/xlnet-smiles\checkpoint-20000\pytorch_model.bin


ValueError: Trainer: evaluation requires an eval_dataset.